In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [28]:
# Load the data
df = pd.read_csv('labeled-data.csv')

# Remove examples labeled 'Done'
df_filtered = df[df['label'] != 'Done']

# Extract features and labels
# Only include features F3,F4,C3,Cz,C4,P3,P4
X = df_filtered.iloc[:, 2:9].values
y = df_filtered['label'].values

In [29]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels for binary classification (N vs. Non-N)
label_encoder_binary = LabelEncoder()
y_encoded_binary = label_encoder_binary.fit_transform(df_filtered['label'] == 'N')

# Convert labels to categorical
y_one_hot_binary = to_categorical(y_encoded_binary, num_classes=2)

In [30]:
# Split the data into train and test sets for binary classification
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(
    X, y_one_hot_binary, test_size=0.1, random_state=42
)

# Reshape the data for Conv1D layer
X_train_binary = X_train_binary.reshape(X_train_binary.shape[0], X_train_binary.shape[1], 1)
X_test_binary = X_test_binary.reshape(X_test_binary.shape[0], X_test_binary.shape[1], 1)

In [31]:
# Build the CNN model for binary classification
# 1 CNN & 1 Dense & 2 output
model_binary = Sequential()
model_binary.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_binary.shape[1], 1)))
model_binary.add(MaxPooling1D(pool_size=2))
model_binary.add(Flatten())
model_binary.add(Dense(64, activation='relu'))
model_binary.add(Dense(2, activation='softmax'))

In [32]:
# Compile the model
model_binary.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_binary.fit(X_train_binary, y_train_binary, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set for binary classification
accuracy_binary = model_binary.evaluate(X_test_binary, y_test_binary)[1]
print(f'Binary Classification Test Accuracy: {accuracy_binary}')

Epoch 1/10
34/34 [==============================] - 0s 4ms/step - loss: 0.6830 - accuracy: 0.6509 - val_loss: 0.6653 - val_accuracy: 0.6556
Epoch 2/10
34/34 [==============================] - 0s 1ms/step - loss: 0.6523 - accuracy: 0.6519 - val_loss: 0.6562 - val_accuracy: 0.6556
Epoch 3/10
34/34 [==============================] - 0s 1ms/step - loss: 0.6472 - accuracy: 0.6519 - val_loss: 0.6500 - val_accuracy: 0.6556
Epoch 4/10
34/34 [==============================] - 0s 1ms/step - loss: 0.6462 - accuracy: 0.6519 - val_loss: 0.6457 - val_accuracy: 0.6556
Epoch 5/10
34/34 [==============================] - 0s 1ms/step - loss: 0.6452 - accuracy: 0.6519 - val_loss: 0.6417 - val_accuracy: 0.6556
Epoch 6/10
34/34 [==============================] - 0s 1ms/step - loss: 0.6442 - accuracy: 0.6537 - val_loss: 0.6380 - val_accuracy: 0.6630
Epoch 7/10
34/34 [==============================] - 0s 1ms/step - loss: 0.6436 - accuracy: 0.6537 - val_loss: 0.6370 - val_accuracy: 0.6630
Epoch 8/10
34/34 [==

In [33]:
# Results:
# Accuracy is higher for L vs R classification than N vs (L or R). 
# Using same 1 CNN & 1 Dense Layer NN